<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align = "center"> Spark Fundamentals I - Introduction to Spark</h1>
<h2 align = "center"> Scala - Working with Scala Libraries</h2>
<br align = "left">

**Related free online courses:**

Related courses can be found in the following learning paths:

- [Spark Fundamentals path](http://cocl.us/Spark_Fundamentals_Path)
- [Big Data Fundamentals path](http://cocl.us/Big_Data_Fundamentals_Path)

<img src="http://spark.apache.org/images/spark-logo.png" height=100>

# Note: Using python cause Scala sucks


## Creating a Spark application using Spark SQL

Spark SQL provides the ability to write relational queries to be run on Spark. There is the abstraction SchemaRDD which is to create an RDD in which you can run SQL, HiveQL, and Scala. In this lab section, you will use SQL to find out the average weather and precipitation for a given time period in New York. The purpose is to demonstrate how to use the Spark SQL libraries on Spark.

### Please note that in Spark 1.3 DataFrames have replaced schemaRDDs however, it is still possible to switch between the two for supporting legacy systems. DataFrames is the recommended method going forward

## Disabled as the file needed is commited

In [ ]:
# download module to run shell commands within this notebook
# import sys.process._

In [ ]:
# download data from IBM Servier
# this may take ~30 seconds depending on your internet speed
# "wget --quiet https://ibm.box.com/shared/static/j8skrriqeqw66f51iyz911zyqai64j2g.zip" !

# println("Data Downloaded!")

Let's unzip the data that we just downloaded into a directory dedicated for this course. Let's choose the directory **/resources/jupyter/labs/BD0211EN/**.

In [ ]:
# unzip the folder's content into "resources" directory
# "unzip -q -o -d /resources/jupyterlab/labs/BD0211EN/ j8skrriqeqw66f51iyz911zyqai64j2g.zip" !

# println("Data Extracted!")

The data is in a folder called **LabData**. Let's list all the files in the data that we just downloaded and extracted.

In [4]:
# list the extracted files
!ls -1 .

BD0211EN-Exercise-Dataframes-py.ipynb
BD0211EN-Exercise-Getting-Started-py-v2.0.ipynb
BD0211EN-Exercise-RDD-py-v2.0.ipynb
BD0211EN-Exercise-Scala-Libraries-sc-v2.0.ipynb
README.md
data
j8skrriqeqw66f51iyz911zyqai64j2g.zip
mtcars.csv
notebook.log
pom.xml


# Set up python environment

In [1]:
!pip install findspark

In [2]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext.getOrCreate()
sc.version

'2.4.4'

# SQL Context

Let's take a look at the nycweather data. So run the following code:

In [38]:
weatherRdd = sc.textFile("nycweather.csv").map(lambda l: l.split(","))
weatherRdd.take(10)


[['"2013-01-01"', '1', '0'],
 ['"2013-01-02"', '-2', '0'],
 ['"2013-01-03"', '-2', '0'],
 ['"2013-01-04"', '1', '0'],
 ['"2013-01-05"', '3', '0'],
 ['"2013-01-06"', '4', '0'],
 ['"2013-01-07"', '5', '0'],
 ['"2013-01-08"', '6', '0'],
 ['"2013-01-09"', '7', '0'],
 ['"2013-01-10"', '7', '0']]

There are three columns in the dataset, the date, the mean temperature in Celsius, and the precipitation for the day. Since we already know the schema, we will infer the schema using reflection.

You will first need to define the SparkSQL context. Do so by creating it from an existing SparkContext. Type in:

In [39]:
sqlContext = pyspark.sql.SQLContext(sc)

Convert RDD to `Row` objects for each row

In [40]:
weatherRdd.map(lambda r: f"{r[0]} {r[1]} {r[2]}").take(10)

['"2013-01-01" 1 0',
 '"2013-01-02" -2 0',
 '"2013-01-03" -2 0',
 '"2013-01-04" 1 0',
 '"2013-01-05" 3 0',
 '"2013-01-06" 4 0',
 '"2013-01-07" 5 0',
 '"2013-01-08" 6 0',
 '"2013-01-09" 7 0',
 '"2013-01-10" 7 0']

In [41]:
weatherRows = weatherRdd.map(lambda r: pyspark.sql.Row(date=r[0], temp=r[1], precipitation=r[2]))
weatherRows.take(10)

[Row(date='"2013-01-01"', precipitation='0', temp='1'),
 Row(date='"2013-01-02"', precipitation='0', temp='-2'),
 Row(date='"2013-01-03"', precipitation='0', temp='-2'),
 Row(date='"2013-01-04"', precipitation='0', temp='1'),
 Row(date='"2013-01-05"', precipitation='0', temp='3'),
 Row(date='"2013-01-06"', precipitation='0', temp='4'),
 Row(date='"2013-01-07"', precipitation='0', temp='5'),
 Row(date='"2013-01-08"', precipitation='0', temp='6'),
 Row(date='"2013-01-09"', precipitation='0', temp='7'),
 Row(date='"2013-01-10"', precipitation='0', temp='7')]

Turn RDD into data frame and register as table

In [42]:
weatherDf = sqlContext.createDataFrame(weatherRows)
sqlContext.registerDataFrameAsTable(weatherDf, "weather")

At this point, you are ready to create and run some queries on the DatatFrame. You want to get a list of the hottest dates with some precipitation. Type in:

In [43]:
df = sqlContext.sql("SELECT * FROM weather WHERE precipitation > 0.0 ORDER BY temp DESC")
df.collect()

[Row(date='"2013-01-30"', precipitation='1.02', temp='9'),
 Row(date='"2013-03-31"', precipitation='2.03', temp='9'),
 Row(date='"2013-11-27"', precipitation='50.29', temp='9'),
 Row(date='"2013-01-14"', precipitation='2.29', temp='8'),
 Row(date='"2013-01-31"', precipitation='22.86', temp='8'),
 Row(date='"2013-01-12"', precipitation='0.51', temp='7'),
 Row(date='"2013-04-12"', precipitation='16', temp='7'),
 Row(date='"2013-01-11"', precipitation='13.97', temp='6'),
 Row(date='"2013-01-29"', precipitation='1.52', temp='6'),
 Row(date='"2013-02-27"', precipitation='39.62', temp='6'),
 Row(date='"2013-11-12"', precipitation='0.76', temp='6'),
 Row(date='"2013-02-19"', precipitation='3.81', temp='5'),
 Row(date='"2013-02-24"', precipitation='0.25', temp='5'),
 Row(date='"2013-02-11"', precipitation='12.45', temp='4'),
 Row(date='"2013-02-13"', precipitation='0.76', temp='4'),
 Row(date='"2013-02-23"', precipitation='6.6', temp='4'),
 Row(date='"2013-02-26"', precipitation='3.56', temp='

## Creating a Spark application using MLlib

In this section, Spark will be used to acquire the K-Means clustering for drop-off latitudes and longitudes of taxis for 3 clusters. The sample data contains a subset of taxi trips with hack license, medallion, pickup date/time, drop off date/time, pickup/drop off latitude/longitude, passenger count, trip distance, trip time and other information. As such, this may give a good indication of where to best to hail a cab.

Remember, this is only a subset of the file that you used in a previous exercise. If you ran this exercise on the full dataset, it would take a long time as we are only running on a test environment with limited resources.

Import the needed packages for K-Means algorithm and Vector packages:

In [ ]:
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.mllib.linalg.Vectors

Create an RDD

In [ ]:
val taxiFile = sc.textFile("/resources/jupyterlab/labs/BD0211EN/LabData/nyctaxisub.csv")

Determine the number of rows in taxiFile.

In [ ]:
taxiFile.count()

Cleanse the data.

In [ ]:
val taxiData=taxiFile.filter(_.contains("2013")).
    filter(_.split(",")(3)!="" ).    //dropoff_latitude
    filter(_.split(",")(4)!="")      //dropoff_longitude

The first filter limits the rows to those that occurred in the year 2013. This will also remove any header in the file. The third and fourth columns contain the drop off latitude and longitude. The transformation will throw exceptions if these values are empty.

Do another count to see what was removed.

In [ ]:
taxiData.count()

In this case, if we had used the full set of data, it would have filtered out a great many more lines.

To fence the area roughly to New York City use this command:

In [ ]:
val taxiFence=taxiData.
    filter(_.split(",")(3).toDouble>40.70).
    filter(_.split(",")(3).toDouble<40.86).
    filter(_.split(",")(4).toDouble>(-74.02)).
    filter(_.split(",")(4).toDouble<(-73.93))

Determine how many are left in taxiFence:

In [ ]:
taxiFence.count()

Approximately, 43,354 rows were dropped since these drop-off points are outside of New York City.

Create Vectors with the latitudes and longitudes that will be used as input to the K-Means algorithm.

In [ ]:
val taxi=taxiFence.
    map{
        line=>Vectors.dense(
            line.split(',').slice(3,5).map(_ .toDouble)
        )
    }

In [ ]:
val iterationCount=10
val clusterCount=3

val model=KMeans.train(taxi,clusterCount,iterationCount)
val clusterCenters=model.clusterCenters.map(_.toArray)

clusterCenters.foreach(lines=>println(lines(0),lines(1)))

Now we know the map co-ordinates. Not surprisingly, the second point is between the Theater District and Grand Central. The third point is in The Village, NYU, Soho and Little Italy area. The first point is the Upper East Side, presumably where people are more likely to take cabs than subways.



## Creating a Spark application using Spark Streaming

This section focuses on Spark Streams, an easy to build, scalable, stateful (e.g. sliding windows) stream processing library. Streaming jobs are written the same way Spark batch jobs are coded and support Java, Scala and Python. In this exercise, taxi trip data will be streamed using a socket connection and then analyzed to provide a summary of number of passengers by taxi vendor. This will be implemented in the Spark shell using Scala.

There are two relevant files for this section. The first one is the `nyctaxi100.csv` which will serve as the source of the stream. The other file is a python file, `taxistreams.py`, which will feed the csv file through a socket connection to simulate a stream.

Run on command line: `python taxistreams.py`
Verify data is being sent: `nc 127.0.0.1 7777`

Once started, the program will bind and listen to the localhost socket 7777. When a connection is made, it will read ‘nyctaxi100.csv’ and send across the socket. The sleep is set such that one line will be sent every 0.5 seconds, or 2 rows a second. This was intentionally set to a high value to make it easier to view the data during execution.

Create the StreamingContext by using the existing SparkContext (sc). It will be using a 1 second batch interval, which means the stream is divided to 1 second batches and each batch becomes a RDD. This is intentional to make it easier to read the data during execution.

In [3]:
from pyspark.streaming import StreamingContext
ssc = StreamingContext(sc, 1)

Create the socket stream that connects to the localhost socket 7777. This matches the port that the Python script is listening on. Each batch from the Stream be a lines RDD.

In [4]:
taxiLines = ssc.socketTextStream("127.0.0.1", 7777)

Next, put in the business logic to split up the lines on each comma and mapping pass(15), which is the vendor, and pass(7), which is the passenger count. Then this is reduced by key resulting in a summary of number of passengers by vendor.

In [7]:
passCount = taxiLines \
    .map(lambda l: l.split(",")) \
    .map(lambda p: (p[15], int(p[7]))) \
    .reduceByKey(lambda a, b: a + b)

Print out to the console:

In [8]:
passCount.pprint()

The next two line starts the stream. 

In [9]:
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2019-12-29 09:15:57
-------------------------------------------

-------------------------------------------
Time: 2019-12-29 09:15:58
-------------------------------------------

-------------------------------------------
Time: 2019-12-29 09:15:59
-------------------------------------------

-------------------------------------------
Time: 2019-12-29 09:16:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-29 09:16:01
-------------------------------------------

-------------------------------------------
Time: 2019-12-29 09:16:02
-------------------------------------------

-------------------------------------------
Time: 2019-12-29 09:16:03
-------------------------------------------

-------------------------------------------
Time: 2019-12-29 09:16:04
-------------------------------------------

-------------------------------------------
Time: 2019-12-29 09:16:05
----------

KeyboardInterrupt: 

It will take a few cycles for the connection to be recognized, and then the data is sent. In this case, 2 rows per second of taxi trip data is receive in a 1 second batch interval.

In the Python terminal, the contents of the file are printed as they are streamed.

**Note: TO STOP THE STREAM PLEASE INTERRUPT THE KERNEL IN BOTH THE OTHER PYTHON NOTEBOOK AND THIS NOTEBOOK. THEN RESTART THIS NOTEBOOK'S KERNEL TO CONTINUE ONTO THE GRAPHX APPLICATION**

This is just a simple example showing how you can take streaming data into Spark and do some type of processing on it. In the case here, the taxi and the number of passengers was extracted from the data stream.

## Creating a Spark application using GraphX

Users.txt is a set of users and followers is the relationship between the users. Take a look at the contents of these two files.

In [ ]:
println("Users: ")
println(scala.io.Source.fromFile("/resources/jupyterlab/labs/BD0211EN/LabData/users.txt").mkString)

println("Followers: ")
println(scala.io.Source.fromFile("/resources/jupyterlab/labs/BD0211EN/LabData/followers.txt").mkString)

Import the GraphX package:

In [ ]:
import org.apache.spark.graphx._

Create the users RDD and parse into tuples of user id and attribute list:

In [ ]:
val users = (sc.textFile("/resources/jupyterlab/labs/BD0211EN/LabData/users.txt").map(line => line.split(",")).map(parts => (parts.head.toLong, parts.tail)))

users.take(5).foreach(println)

Parse the edge data, which is already in userId -> userId format

In [ ]:
val followerGraph = GraphLoader.edgeListFile(sc, "/resources/jupyterlab/labs/BD0211EN/LabData/followers.txt")

Attach the user attributes

In [ ]:
val graph = followerGraph.outerJoinVertices(users) {
    case (uid, deg, Some(attrList)) => attrList
    case (uid, deg, None) => Array.empty[String]
}

Restrict the graph to users with usernames and names:

In [ ]:
val subgraph = graph.subgraph(vpred = (vid, attr) => attr.size == 2)

Compute the PageRank

In [ ]:
val pagerankGraph = subgraph.pageRank(0.001)

Get the attributes of the top pagerank users

In [ ]:
val userInfoWithPageRank = subgraph.outerJoinVertices(pagerankGraph.vertices) {
    case (uid, attrList, Some(pr)) => (pr, attrList.toList)
    case (uid, attrList, None) => (0.0, attrList.toList)
}

Print the line out:

In [ ]:
println(userInfoWithPageRank.vertices.top(5)(Ordering.by(_._2._1)).mkString("\n"))

<div class="alert alert-success alertsuccess" style="margin-top: 20px">
    <strong>Tip</strong>: Enjoyed using Jupyter notebooks with Spark? Get yourself a free 
    <a href="http://cocl.us/DSX_on_Cloud">IBM Cloud</a> account where you can use Data Science Experience notebooks
    and have <em>two</em> Spark executors for free!
</div>

## Summary

Having completed this exercise, you should have some familiarity with using the Spark libraries. In particular, you use Spark SQL to effectively query data inside of Spark. You used Spark Streaming to process incoming streams of batch data. You used Spark's MLlib to compute the *k*-means algorithm to find the best place to hail a cab. Finally, you used Spark's GraphX library to perform and parallel graph calculations on a dataset to find the attributes of the top users.

This notebook is part of the free course on **Cognitive Class** called *Spark Fundamentals I*. If you accessed this notebook outside the course, you can take this free self-paced course, online by going to: http://cocl.us/Spark_Fundamentals_I

### About the Authors:  
Hi! It's Alex Aklson, one of the authors of this notebook. I hope you found this lab educational! There is much more to learn about Spark but you are well on your way. Feel free to connect with me if you have any questions.
<hr>